In [1]:
import pandas as pd

In [7]:
data_stk = {
    "Model": [
        "Static NB", "Static LR", "Static DT", "Static RF", "Static XGB", "Static MLP",
        "Slope-based NB", "Slope-based LR", "Slope-based DT", "Slope-based RF", "Slope-based XGB", "Slope-based MLP",
        "Longitudinal NB", "Longitudinal LR", "Longitudinal DT", "Longitudinal RF", "Longitudinal XGB", "Longitudinal MLP"
    ],
    "Sensitivity": [0.581, 0.507, 0.184, 0.199, 0.111, 0.228,
                    0.875, 0.477, 0.206, 0.228, 0.081, 0.206,
                    0.824, 0.338, 0.214, 0.273, 0.096, 0.169],
    "F1": [0.049, 0.064, 0.040, 0.045, 0.049, 0.068,
           0.045, 0.062, 0.042, 0.043, 0.041, 0.061,
           0.048, 0.063, 0.044, 0.052, 0.045, 0.050],
    "F2": [0.107, 0.133, 0.074, 0.084, 0.072, 0.117,
           0.105, 0.130, 0.080, 0.084, 0.057, 0.105,
           0.110, 0.122, 0.084, 0.101, 0.065, 0.086],
    "AUC": [0.567, 0.602, 0.514, 0.530, 0.576, 0.597,
            0.530, 0.607, 0.528, 0.531, 0.589, 0.586,
            0.562, 0.611, 0.561, 0.534, 0.567, 0.591]
}

data_stroke = pd.DataFrame(data_stk)

# --- Rank Fusion (Reciprocal Rank Fusion) ---
metrics = ["F1", "F2", "AUC"]
k = 0 
for m in metrics:
    data_stroke[f"{m}_rank"] = data_stroke[m].rank(ascending=False, method="min")

# Compute reciprocal rank score
data_stroke["RRF_score"] = data_stroke[[f"{m}_rank" for m in metrics]].apply(
    lambda row: sum(1.0 / (k + row[r]) for r in row.index),
    axis=1
)

# Final ranking
data_stroke["Final_rank"] = data_stroke["RRF_score"].rank(ascending=False, method="min")


data_stroke[["Model", "Sensitivity", "F1", "F2", "AUC", "RRF_score", "Final_rank"]].sort_values("Final_rank")

,Model,Sensitivity,F1,F2,AUC,RRF_score,Final_rank
1,Static LR,0.507,0.064,0.133,0.602,1.833333,1.0
13,Longitudinal LR,0.338,0.063,0.122,0.611,1.666667,2.0
5,Static MLP,0.228,0.068,0.117,0.597,1.500000,3.0
7,Slope-based LR,0.477,0.062,0.130,0.607,1.250000,4.0
11,Slope-based MLP,0.206,0.061,0.105,0.586,0.485714,5.0
17,Longitudinal MLP,0.169,0.050,0.086,0.591,0.442857,6.0
0,Static NB,0.581,0.049,0.107,0.567,0.402778,7.0
12,Longitudinal NB,0.824,0.048,0.110,0.562,0.390909,8.0
15,Longitudinal RF,0.273,0.052,0.101,0.534,0.354701,9.0
4,Static XGB,0.111,0.049,0.072,0.576,0.312500,10.0


In [10]:

data_death = {
    "Model": [
        "Static NB", "Static LR", "Static DT", "Static RF", "Static XGB", "Static MLP",
        "Longitudinal NB", "Longitudinal LR", "Longitudinal DT", "Longitudinal RF", "Longitudinal XGB", "Longitudinal MLP",
        "All-features NB", "All-features LR", "All-features DT", "All-features RF", "All-features XGB", "All-featues MLP"
    ],
    "F1": [0.161, 0.177, 0.162, 0.191, 0.194, 0.172,
           0.131, 0.181, 0.153, 0.183, 0.195, 0.172,
           0.134, 0.180, 0.166, 0.184, 0.192, 0.159],
    "F2": [0.305, 0.331, 0.303, 0.347, 0.348, 0.308,
           0.268, 0.337, 0.294, 0.337, 0.350, 0.308,
           0.272, 0.334, 0.310, 0.339, 0.346, 0.288],
    "AUC": [0.718, 0.774, 0.691, 0.780, 0.777, 0.734,
            0.712, 0.777, 0.695, 0.779, 0.785, 0.731,
            0.708, 0.774, 0.690, 0.753, 0.779, 0.723]
}

df_death = pd.DataFrame(data_death)

# --- Rank Fusion (Reciprocal Rank Fusion) ---
metrics = ["F1", "F2", "AUC"]
k = 0  # reciprocal rank parameter (can change if needed, e.g. 60)

# Rank for each metric (higher is better)
for m in metrics:
    df_death[f"{m}_rank"] = df_death[m].rank(ascending=False, method="min")

# Compute reciprocal rank fusion score
df_death["RRF_score"] = df_death[[f"{m}_rank" for m in metrics]].apply(
    lambda row: sum(1.0 / (k + row[r]) for r in row.index),
    axis=1
)

# Final fused rank
df_death["Final_rank"] = df_death["RRF_score"].rank(ascending=False, method="min")

# Show sorted results
df_death[["Model", "F1", "F2", "AUC", "RRF_score", "Final_rank"]].sort_values("Final_rank")


,Model,F1,F2,AUC,RRF_score,Final_rank
10,Longitudinal XGB,0.195,0.350,0.785,3.000000,1.0
4,Static XGB,0.194,0.348,0.777,1.200000,2.0
3,Static RF,0.191,0.347,0.780,1.083333,3.0
16,All-features XGB,0.192,0.346,0.779,0.916667,4.0
9,Longitudinal RF,0.183,0.337,0.779,0.666667,5.0
15,All-features RF,0.184,0.339,0.753,0.511111,6.0
7,Longitudinal LR,0.181,0.337,0.777,0.509524,7.0
13,All-features LR,0.180,0.334,0.774,0.392857,8.0
1,Static LR,0.177,0.331,0.774,0.365079,9.0
5,Static MLP,0.172,0.308,0.734,0.290909,10.0


In [11]:

data_cvdeath = {
    "Model": [
        "Static NB", "Static LR", "Static DT", "Static RF", "Static XGB", "Static MLP",
        "Longitudinal NB", "Longitudinal LR", "Longitudinal DT", "Longitudinal RF", "Longitudinal XGB", "Longitudinal MLP",
        "All-features NB", "All-features LR", "All-features DT", "All-features RF", "All-features XGB", "All-features MLP"
    ],
    "F1": [0.129, 0.146, 0.102, 0.172, 0.177, 0.152,
           0.113, 0.147, 0.108, 0.140, 0.168, 0.148,
           0.115, 0.151, 0.120, 0.173, 0.157, 0.149],
    "F2": [0.248, 0.275, 0.188, 0.280, 0.300, 0.270,
           0.234, 0.279, 0.201, 0.247, 0.282, 0.260,
           0.237, 0.271, 0.218, 0.285, 0.269, 0.265],
    "AUC": [0.729, 0.760, 0.651, 0.774, 0.767, 0.731,
            0.723, 0.766, 0.634, 0.710, 0.766, 0.731,
            0.722, 0.727, 0.641, 0.764, 0.750, 0.734]
}

df_cvdeath = pd.DataFrame(data_cvdeath)

# --- Rank Fusion (Reciprocal Rank Fusion) ---
metrics = ["F1", "F2", "AUC"]
k = 0  # reciprocal rank parameter

# Rank for each metric (higher is better)
for m in metrics:
    df_cvdeath[f"{m}_rank"] = df_cvdeath[m].rank(ascending=False, method="min")

# Compute reciprocal rank fusion score
df_cvdeath["RRF_score"] = df_cvdeath[[f"{m}_rank" for m in metrics]].apply(
    lambda row: sum(1.0 / (k + row[r]) for r in row.index),
    axis=1
)

# Final fused rank
df_cvdeath["Final_rank"] = df_cvdeath["RRF_score"].rank(ascending=False, method="min")

# Show sorted results
df_cvdeath[["Model", "F1", "F2", "AUC", "RRF_score", "Final_rank"]].sort_values("Final_rank")


,Model,F1,F2,AUC,RRF_score,Final_rank
4,Static XGB,0.177,0.300,0.767,2.500000,1.0
3,Static RF,0.172,0.280,0.774,1.583333,2.0
15,All-features RF,0.173,0.285,0.764,1.200000,3.0
10,Longitudinal XGB,0.168,0.282,0.766,0.916667,4.0
7,Longitudinal LR,0.147,0.279,0.766,0.633333,5.0
16,All-features XGB,0.157,0.269,0.750,0.453968,6.0
1,Static LR,0.146,0.275,0.760,0.424242,7.0
5,Static MLP,0.152,0.270,0.731,0.402778,8.0
13,All-features LR,0.151,0.271,0.727,0.369048,9.0
17,All-features MLP,0.149,0.265,0.734,0.350000,10.0


In [13]:
import pandas as pd

# Data
data_hf = {
    "Model": [
        "Static NB", "Static LR", "Static DT", "Static RF", "Static XGB", "Static MLP",
        "Slope-based NB", "Slope-based LR", "Slope-based DT", "Slope-based RF", "Slope-based XGB", "Slope-based MLP",
        "Longitudinal NB", "Longitudinal LR", "Longitudinal DT", "Longitudinal RF", "Longitudinal XGB", "Longitudinal MLP"
    ],
    "F1": [0.331, 0.357, 0.381, 0.386, 0.412, 0.335,
           0.326, 0.377, 0.389, 0.371, 0.388, 0.341,
           0.326, 0.368, 0.385, 0.369, 0.404, 0.348],
    "F2": [0.482, 0.542, 0.531, 0.559, 0.570, 0.479,
           0.496, 0.548, 0.537, 0.547, 0.554, 0.487,
           0.498, 0.547, 0.538, 0.546, 0.567, 0.501],
    "AUC": [0.723, 0.772, 0.755, 0.785, 0.806, 0.728,
            0.719, 0.785, 0.760, 0.774, 0.796, 0.731,
            0.716, 0.782, 0.762, 0.758, 0.801, 0.739]
}

df_hf = pd.DataFrame(data_hf)

# --- Rank Fusion (Reciprocal Rank Fusion) ---
metrics = ["F1", "F2", "AUC"]
k = 0  # reciprocal rank parameter

# Rank each metric (higher is better)
for m in metrics:
    df_hf[f"{m}_rank"] = df_hf[m].rank(ascending=False, method="min")

# Compute reciprocal rank fusion score
df_hf["RRF_score"] = df_hf[[f"{m}_rank" for m in metrics]].apply(
    lambda row: sum(1.0 / (k + row[r]) for r in row.index),
    axis=1
)

# Final fused rank
df_hf["Final_rank"] = df_hf["RRF_score"].rank(ascending=False, method="min")

# Show sorted results
df_hf[["Model", "F1", "F2", "AUC", "RRF_score", "Final_rank"]].sort_values("Model")


,Model,F1,F2,AUC,RRF_score,Final_rank
14,Longitudinal DT,0.385,0.538,0.762,0.377778,9.0
13,Longitudinal LR,0.368,0.547,0.782,0.424242,7.0
17,Longitudinal MLP,0.348,0.501,0.739,0.230769,13.0
12,Longitudinal NB,0.326,0.498,0.716,0.185808,16.0
15,Longitudinal RF,0.369,0.546,0.758,0.315909,11.0
16,Longitudinal XGB,0.404,0.567,0.801,1.500000,2.0
8,Slope-based DT,0.389,0.537,0.760,0.524242,6.0
7,Slope-based LR,0.377,0.548,0.785,0.575000,5.0
11,Slope-based MLP,0.341,0.487,0.731,0.205357,14.0
6,Slope-based NB,0.326,0.496,0.719,0.184314,17.0


In [16]:

data_inp = {
    "Model": [
        "Static NB", "Static LR", "Static DT", "Static RF", "Static XGB", "Static MLP",
        "Slope-based NB", "Slope-based LR", "Slope-based DT", "Slope-based RF", "Slope-based XGB", "Slope-based MLP",
        "Longitudinal NB", "Longitudinal LR", "Longitudinal DT", "Longitudinal RF", "Longitudinal XGB", "Longitudinal MLP"
    ],
    "F1": [0.399, 0.430, 0.407, 0.409, 0.418, 0.376,
           0.377, 0.428, 0.391, 0.409, 0.420, 0.379,
           0.376, 0.425, 0.410, 0.409, 0.425, 0.383],
    "F2": [0.552, 0.548, 0.501, 0.587, 0.589, 0.476,
           0.571, 0.552, 0.477, 0.587, 0.590, 0.480,
           0.570, 0.553, 0.505, 0.587, 0.588, 0.484],
    "AUC": [0.646, 0.683, 0.648, 0.627, 0.632, 0.623,
            0.639, 0.679, 0.631, 0.627, 0.670, 0.619,
            0.638, 0.678, 0.649, 0.619, 0.670, 0.630]
}

df_inp = pd.DataFrame(data_inp)

# --- Rank Fusion (Reciprocal Rank Fusion) ---
metrics = ["F1", "F2", "AUC"]
k = 0  # reciprocal rank parameter

# Rank each metric (higher is better)
for m in metrics:
    df_inp[f"{m}_rank"] = df_inp[m].rank(ascending=False, method="min")

# Compute reciprocal rank fusion score
df_inp["RRF_score"] = df_inp[[f"{m}_rank" for m in metrics]].apply(
    lambda row: sum(1.0 / (k + row[r]) for r in row.index),
    axis=1
)

# Final fused rank
df_inp["Final_rank"] = df_inp["RRF_score"].rank(ascending=False, method="min")

# Show sorted results
df_inp[["Model", "Final_rank", "F1", "F2", "AUC", "RRF_score"]].sort_values("Model")


,Model,Final_rank,F1,F2,AUC,RRF_score
14,Longitudinal DT,10.0,0.410,0.505,0.649,0.386447
13,Longitudinal LR,5.0,0.425,0.553,0.678,0.777778
17,Longitudinal MLP,16.0,0.383,0.484,0.630,0.215018
12,Longitudinal NB,14.0,0.376,0.570,0.638,0.283824
15,Longitudinal RF,9.0,0.409,0.587,0.619,0.433824
16,Longitudinal XGB,4.0,0.425,0.588,0.670,0.916667
8,Slope-based DT,15.0,0.391,0.477,0.631,0.219080
7,Slope-based LR,3.0,0.428,0.552,0.679,1.100000
11,Slope-based MLP,17.0,0.379,0.480,0.619,0.187990
6,Slope-based NB,11.0,0.377,0.571,0.639,0.316468


In [20]:

data_acs = {
    "Model": [
        "Static NB", "Static LR", "Static DT", "Static RF", "Static XGB", "Static MLP",
        "Slope Based NB", "Slope Based LR", "Slope Based DT", "Slope Based RF", "Slope Based XGB", "Slope Based MLP",
        "Longitudinal NB", "Longitudinal LR", "Longitudinal DT", "Longitudinal RF", "Longitudinal XGB", "Longitudinal MLP"
    ],
    "F1": [0.064, 0.054, 0.055, 0.053, 0.073, 0.088,
           0.054, 0.056, 0.072, 0.058, 0.086, 0.084,
           0.054, 0.055, 0.053, 0.044, 0.103, 0.092],
    "F2": [0.135, 0.118, 0.097, 0.096, 0.101, 0.139,
           0.118, 0.122, 0.133, 0.104, 0.111, 0.130,
           0.119, 0.121, 0.097, 0.082, 0.138, 0.141],
    "AUC": [0.738, 0.740, 0.596, 0.605, 0.721, 0.692,
            0.721, 0.736, 0.620, 0.623, 0.691, 0.687,
            0.720, 0.737, 0.604, 0.580, 0.715, 0.698]
}

df_acs = pd.DataFrame(data_acs)

# --- Rank Fusion (Reciprocal Rank Fusion) ---
metrics = ["F1", "F2", "AUC"]
k = 0  # reciprocal rank parameter

# Rank each metric (higher is better)
for m in metrics:
    df_acs[f"{m}_rank"] = df_acs[m].rank(ascending=False, method="min")

# Compute reciprocal rank fusion score
df_acs["RRF_score"] = df_acs[[f"{m}_rank" for m in metrics]].apply(
    lambda row: sum(1.0 / (k + row[r]) for r in row.index),
    axis=1
)

# Final fused rank
df_acs["Final_rank"] = df_acs["RRF_score"].rank(ascending=False, method="min")

# Show sorted results
df_acs[["Model","Final_rank", "F1", "F2", "AUC", "RRF_score"]].sort_values("Model")


,Model,Final_rank,F1,F2,AUC,RRF_score
14,Longitudinal DT,16.0,0.053,0.097,0.604,0.191667
13,Longitudinal LR,6.0,0.055,0.121,0.737,0.549242
17,Longitudinal MLP,1.0,0.092,0.141,0.698,1.611111
12,Longitudinal NB,13.0,0.054,0.119,0.720,0.330891
15,Longitudinal RF,18.0,0.044,0.082,0.580,0.166667
16,Longitudinal XGB,2.0,0.103,0.138,0.715,1.458333
8,Slope Based DT,11.0,0.072,0.133,0.620,0.414286
7,Slope Based LR,7.0,0.056,0.122,0.736,0.492857
11,Slope Based MLP,8.0,0.084,0.130,0.687,0.450000
6,Slope Based NB,12.0,0.054,0.118,0.721,0.376923
